In [160]:
import pandas as pd
from pandas import DataFrame, Series
import copy
from string import punctuation
import re
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
from nltk import word_tokenize

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\carol\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [161]:
comments = pd.read_csv("C:/Users/carol/Downloads/comments.csv")
models = pd.read_csv("C:/Users/carol/Downloads/models.csv")
models.columns = ['brand','model']

In [162]:
x = copy.deepcopy(comments)
comments_copy = x.dropna().reset_index(drop=True).drop(columns='Unnamed: 0')

In [163]:
def remove_space(s):
    return s.replace("\n"," ")

comments_copy['comments'] = comments_copy['comments'].apply(remove_space)

In [164]:
def removepunc(item):
    for p in punctuation:
        item = item.replace(p,'')
    return item

def lowerize(x):
    return x.lower()

comments_copy['comments_clean'] = comments_copy['comments'].apply(removepunc).apply(lowerize)

In [165]:
models['brand'] = models['brand'].apply(removepunc)

def model_to_brand(s):
    for i in models.index.values:
        s = s.replace(models["model"][i].lower(),models["brand"][i].lower())
    return s

comments_copy['comments_model_replace'] = comments_copy['comments_clean'].apply(model_to_brand)

In [166]:
temp = copy.deepcopy(comments_copy)

In [167]:
temp['comments_appear'] = temp['comments_model_replace'].apply(word_tokenize).apply(set).apply(list)

# Remove stop words
stop_words = set(stopwords.words('english'))
def remove_stopwords(s):
    return [w for w in s if not w in stop_words] 
    
temp['final_comments'] =  temp['comments_appear'].apply(remove_stopwords)
#filtered_sentence = [w for w in temp['comments_appear'] if not w in stop_words]

In [168]:
count=[]
for i in range(len(temp)):
    count+=temp['final_comments'].loc[i]

In [169]:
word_freq = nltk.FreqDist(count)

In [170]:
models_unique = models['brand'].drop_duplicates().tolist()

In [171]:
top_words = word_freq.most_common(500)
top_brands = []
for (key, items) in top_words:
    if key in models_unique:
        model_counts = (key,items)
        top_brands.append(model_counts)

In [172]:
top_brands

[('bmw', 1435),
 ('audi', 1246),
 ('acura', 1170),
 ('hyundai', 750),
 ('volkswagen', 653),
 ('honda', 632),
 ('mercedes', 450),
 ('infiniti', 437),
 ('ford', 429),
 ('kia', 400),
 ('cadillac', 372),
 ('chrysler', 363),
 ('subaru', 236),
 ('lexus', 198),
 ('nissan', 135),
 ('toyota', 125),
 ('volvo', 115),
 ('chevrolet', 103),
 ('gm', 97)]

### Lift and MDS

First calculate Lift counting any co-occurence at one row as one from whole comments

In [173]:
import itertools

In [174]:
brand_list = models['brand'].unique()
y = list(itertools.product(brand_list,repeat=2))
z = DataFrame(index=range(len(y)), data={'pairs':y, 'count':0*len(y)})
a = DataFrame(index=range(len(brand_list)), data={'brands':brand_list, 'count':0*len(y)})

In [175]:
def brand_detect(d):
    bd = []
    for x in d:
        if x in brand_list:
            brand_mentioned = x
            bd.append(brand_mentioned)
    return bd

def brand_combination(l):
    return list(itertools.combinations(l,2))

temp['pairs'] = temp['final_comments'].apply(brand_detect).apply(brand_combination)

In [176]:
pair_list =[]

for i in range(len(temp)):
    pair_list+= temp['pairs'][i]

In [177]:
for t in range(len(z)):
    p_count = pair_list.count(z['pairs'][t])
    z['count'][t] = p_count

C:\Users\carol\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [178]:
for i in range(len(a)):
    b_count = count.count(a['brands'][i])
    a['count'][i] = b_count
    
N=len(temp)

C:\Users\carol\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [179]:
#lift matrrix of whole brands
n = len(a)
mtx = [[0]*n for i in range(n)]
N = len(temp)
for i in range(n):
    for j in range(n):
        mtx[i][j] = (N*(z['count'].loc[(i*n+j)]))/((a['count'].loc[i])*(a['count'].loc[j]))

C:\Users\carol\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in longlong_scalars
  import sys


In [180]:
#Visualize in DataFrame
similarities = DataFrame(data=mtx, index=brand_list, columns=brand_list)
print(similarities)

                acura  alfaromeo      audi  bentley       bmw  bugatti  \
acura        0.000000   0.000000  0.051436      NaN  0.041684      NaN   
alfaromeo    0.993639   0.000000  0.093303      NaN  0.972174      NaN   
audi         1.162462   0.279910  0.000000      NaN  0.436152      NaN   
bentley           NaN        NaN       NaN      NaN       NaN      NaN   
bmw          0.905147   0.000000  0.886283      NaN  0.000000      NaN   
bugatti           NaN        NaN       NaN      NaN       NaN      NaN   
buick        1.608527   0.000000  0.094401      NaN  0.819676      NaN   
cadillac     1.458673   0.000000  1.251066      NaN  1.039468      NaN   
chevrolet    0.995569   0.000000  1.129603      NaN  0.135286      NaN   
chrysler     0.400193   1.601320  3.271525      NaN  0.441451      NaN   
dodge        0.681147   1.684867  0.872182      NaN  1.110721      NaN   
ford         0.507937   0.000000  0.000000      NaN  0.000000      NaN   
gm           1.101198   0.000000  0.95

lift matrix of top 10 brands

In [181]:
top_brand_l=[]
for (key,value) in top_brands:
    top_brand_l.append(key)

top_brand_list = top_brand_l[0:10]

In [204]:
d = list(itertools.product(top_brand_list,repeat=2))
e = DataFrame(index=range(len(d)), data={'pairs':d, 'count':0*len(d)})
f = DataFrame(index=range(len(top_brand_list)), data={'brands':top_brand_list, 'count':0*len(top_brand_list)})

In [206]:
def brand_detect(d):
    bd = []
    for x in d:
        if x in top_brand_list:
            brand_mentioned = x
            bd.append(brand_mentioned)
    return bd

temp['top_pairs'] = temp['final_comments'].apply(brand_detect).apply(brand_combination)

In [207]:
top_pair_list =[]

for i in range(len(temp)):
    top_pair_list+= temp['top_pairs'][i]

In [208]:
for t in range(len(e)):
    tp_count = top_pair_list.count(e['pairs'][t])
    e['count'][t] = tp_count

C:\Users\carol\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [227]:
i = g[g['pairs']==('kia', 'infiniti')].index.values
j = g[g['pairs']==('infiniti', 'kia')].index.values
g['count'].loc[i]= (g['count'].loc[i]+g['count'].loc[j])
g['count'].loc[j]= (g['count'].loc[i]+g['count'].loc[j])

In [232]:
e['count'].loc[1]

317

In [229]:
import copy
g = copy.deepcopy(e)

for (b1,b2) in g['pairs']:
    i = e[e['pairs']==(b1, b2)].index.values
    j = e[e['pairs']==(b2, b1)].index.values
    g['count'].loc[i]= e['count'].loc[i]+e['count'].loc[j]
    g['count'].loc[j]= e['count'].loc[i]+e['count'].loc[j]

C:\Users\carol\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [219]:
g[g['pairs']==('bmw','audi')]['count']#+g['count'][g['pairs']=='('audi','bmw')']

1   NaN
Name: count, dtype: float64

In [186]:
for i in range(len(f)):
    tb_count = count.count(f['brands'][i])
    f['count'][i] = tb_count
    
N=len(temp)

C:\Users\carol\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [187]:
#lift matrix of top brands
tn = len(f)
tmtx = [[0]*tn for i in range(tn)]
for i in range(tn):
    for j in range(tn):
        tmtx[i][j] = (N*(e['count'].loc[(i*tn+j)]))/((f['count'].loc[i])*(f['count'].loc[j]))

In [188]:
tsimilarities = DataFrame(data=tmtx, index=top_brand_list, columns=top_brand_list)
tsimilarities

,bmw,audi,acura,hyundai,volkswagen,honda,mercedes,infiniti,ford,kia
bmw,0.000000,0.886283,0.905147,0.306559,0.634841,0.518134,0.000000,1.363157,1.226171,0.296108
audi,0.436152,0.000000,1.162462,0.663324,1.370114,0.755431,0.722167,1.542386,0.729462,1.785357
acura,0.041684,0.051436,0.000000,0.017091,0.000000,0.000000,0.075958,0.527970,0.029879,0.010682
hyundai,0.534156,0.037446,1.931238,0.000000,0.316425,0.000000,0.903523,0.198282,0.590402,0.416583
volkswagen,0.026674,1.050626,0.844061,0.296010,0.000000,0.617765,0.034024,0.648172,0.785173,0.000000
honda,0.391356,0.533246,1.947030,0.558960,0.472408,0.000000,0.492166,2.968441,0.829700,0.474589
mercedes,2.314674,1.381924,1.414722,0.681345,0.986701,0.861290,0.000000,1.906560,1.217058,0.805394
infiniti,0.063773,0.100990,0.889728,0.244040,0.000000,1.212717,0.127104,0.000000,0.559969,0.000000
ford,0.000000,0.000000,0.507937,0.015537,0.000000,0.000000,0.000000,0.159991,0.000000,0.029132
kia,0.426744,1.253762,0.448628,0.483237,5.473637,0.553687,0.055544,0.600566,1.223531,0.000000


Draw MDS Plot-first similarity matrix

In [189]:
import numpy as np
from sklearn import manifold
%pylab inline

Populating the interactive namespace from numpy and matplotlib


C:\Users\carol\Anaconda3\lib\site-packages\IPython\core\magics\pylab.py:160: UserWarning: pylab import has clobbered these variables: ['e', 'seed', 'copy', 'f']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [200]:
dissimilarity_matrix = 1/similarities
np.fill_diagonal(dissimilarity_matrix.values, 1)

tdissimilarity_matrix = 1/tsimilarities
np.fill_diagonal(tdissimilarity_matrix.values, 1)
tdissimilarity_matrix_1=tdissimilarity_matrix.replace(inf,1)

In [201]:
tdissimilarity_matrix_1

,bmw,audi,acura,hyundai,volkswagen,honda,mercedes,infiniti,ford,kia
bmw,1.000000,1.128308,1.104793,3.262016,1.575197,1.930003,1.000000,0.733591,0.815547,3.377146
audi,2.292779,1.000000,0.860243,1.507560,0.729866,1.323747,1.384721,0.648346,1.370874,0.560112
acura,23.989798,19.441488,1.000000,58.511702,1.000000,1.000000,13.165133,1.894045,33.468694,93.618724
hyundai,1.872114,26.705341,0.517803,1.000000,3.160309,1.000000,1.106779,5.043316,1.693760,2.400480
volkswagen,37.489698,0.951814,1.184749,3.378262,1.000000,1.618739,29.390878,1.542800,1.273605,1.000000
honda,2.555215,1.875308,0.513603,1.789037,2.116813,1.000000,2.031835,0.336877,1.205254,2.107088
mercedes,0.432026,0.723629,0.706853,1.467685,1.013479,1.161049,1.000000,0.524505,0.821654,1.241628
infiniti,15.680511,9.902017,1.123939,4.097695,1.000000,0.824595,7.867574,1.000000,1.785814,1.000000
ford,1.000000,1.000000,1.968747,64.362873,1.000000,1.000000,1.000000,6.250350,1.000000,34.326865
kia,2.343326,0.797600,2.229017,2.069379,0.182694,1.806076,18.003601,1.665095,0.817306,1.000000


### From this part, Task E starts

In [326]:
aspiration = pd.read_csv("C:/Users/carol/Downloads/aspiration.csv")
temp2 = copy.deepcopy(temp)

We defined 'aspiring brand' as the brand people want to have but not have possessed yet. We first looked up synonyms of 'aspiring' and manually checked all the possible variation of each synonym.
We choos to use phrases instead of single word because exceptions(for instance, 'leasing dream' doesn't imply 'aspiration').
We also decided not to differentiate each synonym to the others because there is no clear semantic difference among them. Based on following list, we conducted lift analysis. Calculation is quite similar to aforewritten ones.

In [334]:
aspiration

,word,aspr
0,dream car,aspiration
1,dreamed,aspiration
2,dreaming,aspiration
3,dream of,aspiration
4,dream to,aspiration
5,couldn't afford,aspiration
6,wish,aspiration
7,best,aspiration
8,love to have,aspiration
9,wanted to,aspiration


In [327]:
def aspiring(s):
    s = " ".join(str(x) for x in s)
    for i in aspiration['word'].index.values:
        s = s.replace(aspiration['word'][i],aspiration['aspr'][i])
    return s
temp['comments_asp_replace'] = temp['final_comments'].apply(aspiring).apply(word_tokenize)

In [328]:
asp_count = []
for i in range(len(temp)):
    asp_count+=temp['comments_asp_replace'][i]
asp_freq = nltk.FreqDist(count)

In [329]:
aspiring_df = pd.DataFrame(columns = aspiration['aspr'].unique())

def aspiring_mentioned(item):
    if asp in item:
        return 1
    else:
        return 0

for asp in aspiration['aspr'].unique():
    aspiring_df[asp] = temp['comments_asp_replace'].apply(aspiring_mentioned)

In [330]:
top_5_cars = ['bmw','audi','acura','mercedes','cadillac']
asp_df = pd.DataFrame(columns = top_5_cars)

def brand_mentioned(item):
    if brand in item:
        return 1
    else:
        return 0
      
for brand in top_5_cars:
    asp_df[brand] = temp['final_comments'].apply(brand_mentioned)

In [331]:
top_aspiration = aspiration['aspr'].unique()
aspiring_df2=pd.DataFrame(columns = top_aspiration)

for i in range(len(top_5_cars)):
    asp_list = []
    for j in range(len(aspiration['aspr'].unique())):
        lift_brand_aspr = (((asp_df[top_5_cars[i]] + aspiring_df[top_aspiration[j]]) > 1).sum())/(asp_df[top_5_cars[i]].sum()*aspiring_df[top_aspiration[j]].sum())
        lift_brand_aspr = lift_brand_aspr*len(aspiring_df)
        aspiring_df2.loc[top_5_cars[i],top_aspiration[j]] = lift_brand_aspr

In [332]:
print ('Below are the lift ratios between top 5 cars and top aspiring expression \n')
aspiring_df2

Below are the lift ratios between top 5 cars and top aspiring expression 



,aspiration
bmw,1.51357
audi,1.72933
acura,1.47333
mercedes,1.72379
cadillac,1.99256
